In [ ]:
import base64
import io
import pandas as pd
from dash import Dash, dcc, html, Input, Output, State, callback_context
import plotly.express as px
import dash


app = Dash(__name__)


styles = {
    'container': {
        'margin': '20px',
        'fontFamily': 'Arial, sans-serif'
    },
    'header': {
        'backgroundColor': '#1a2d40',
        'padding': '2rem',
        'borderRadius': '15px',
        'color': 'white',
        'marginBottom': '2rem',
        'boxShadow': '0 4px 6px rgba(0,0,0,0.1)'
    },
    'upload': {
        'width': '100%',
        'height': '60px',
        'lineHeight': '60px',
        'borderWidth': '1px',
        'borderStyle': 'dashed',
        'borderRadius': '5px',
        'textAlign': 'center',
        'marginBottom': '2rem'
    },
    'filters': {
        'backgroundColor': 'white',
        'padding': '2rem',
        'borderRadius': '15px',
        'boxShadow': '0 4px 6px rgba(0,0,0,0.1)',
        'marginBottom': '2rem'
    },
    'graph': {
        'backgroundColor': 'white',
        'borderRadius': '15px',
        'padding': '15px',
        'marginBottom': '20px',
        'boxShadow': '0 4px 6px rgba(0,0,0,0.1)'
    },
    'stats': {
        'backgroundColor': '#1a2d40',
        'color': 'white',
        'padding': '1.5rem',
        'borderRadius': '15px',
        'marginTop': '2rem'
    }
}

app.layout = html.Div(style=styles['container'], children=[
    dcc.Store(id='stored-data'),  
    
    html.Div(style=styles['header'], children=[
        html.H1("Панель мониторинга пациентов", style={'textAlign': 'center', 'marginBottom': '10px'}),
        html.P("Интерактивная визуализация медицинских данных в реальном времени", 
              style={'textAlign': 'center', 'fontSize': '1.1rem'})
    ]),
    
    # Компонент для загрузки файла
    dcc.Upload(
        id='upload-data',
        children=html.Div(['Перетащите или ', html.A('выберите CSV-файл')]),
        style=styles['upload'],
        multiple=False
    ),
    
    # Контейнер для основного контента (скрыт до загрузки данных)
    html.Div(id='content', style={'display': 'none'}, children=[
        html.Div(style=styles['filters'], children=[
            html.Div([
                html.Label("Диапазон дат", style={'fontWeight': 'bold', 'marginBottom': '5px'}),
                dcc.DatePickerRange(
                    id='date-range',
                    display_format='YYYY-MM-DD'
                )
            ], style={'width': '100%', 'marginBottom': '20px'}),
            
            html.Div([
                html.Label("Пол пациента", style={'fontWeight': 'bold', 'marginBottom': '5px'}),
                dcc.Dropdown(
                    id="gender-filter",
                    options=[{"label": g, "value": g} for g in ["All", "Male", "Female"]],
                    value="All",
                    clearable=False,
                    style={'borderRadius': '8px'}
                )
            ], style={'width': '30%', 'display': 'inline-block', 'marginRight': '5%'}),
            
            html.Div([
                html.Label("Состояние", style={'fontWeight': 'bold', 'marginBottom': '5px'}),
                dcc.Dropdown(
                    id="condition-filter",
                    options=[{"label": c, "value": c} for c in ["All", "Stable", "Critical", "Recovering"]],
                    value="All",
                    clearable=False,
                    style={'borderRadius': '8px'}
                )
            ], style={'width': '30%', 'display': 'inline-block', 'marginRight': '5%'}),
            
            html.Div([
                html.Label("Возрастной диапазон", style={'fontWeight': 'bold', 'marginBottom': '15px'}),
                dcc.RangeSlider(
                    id="age-slider",
                    min=0,
                    max=100,
                    step=5,
                    marks={i: {'label': str(i), 'style': {'color': '#1a2d40'}} for i in range(0, 101, 10)},
                    value=[0, 100],
                    tooltip={'placement': 'bottom'}
                )
            ], style={'width': '30%', 'display': 'inline-block'})
        ]),
        
        dcc.Loading(
            id="loading",
            type="circle",
            children=[
                html.Div([
                    dcc.Graph(
                        id="vital-signs-plot",
                        style={**styles['graph'], 'height': '500px'}
                    ),
                    
                    html.Div([
                        html.Div(
                            dcc.Graph(id="patient-distribution"),
                            style={**styles['graph'], 'width': '48%', 'display': 'inline-block'}
                        ),
                        html.Div(
                            dcc.Graph(id="hr-distribution"),
                            style={**styles['graph'], 'width': '48%', 'display': 'inline-block', 'float': 'right'}
                        )
                    ]),
                    
                    html.Div(
                        id="advanced-stats",
                        style=styles['stats']
                    )
                ])
            ]
        )
    ])
])

@app.callback(
    [Output('stored-data', 'data'),
     Output('content', 'style'),
     Output('date-range', 'start_date'),
     Output('date-range', 'end_date'),
     Output('age-slider', 'marks')],
    [Input('upload-data', 'contents')],
    [State('upload-data', 'filename')]
)
def update_data(contents, filename):
    if contents is None:
        return dash.no_update, {'display': 'none'}, dash.no_update, dash.no_update, dash.no_update
    
    content_type, content_string = contents.split(',')
    decoded = base64.b64decode(content_string)
    
    try:
        df = pd.read_csv(
            io.StringIO(decoded.decode('utf-8')),
            sep=';',
            parse_dates=['last_update'],
            dayfirst=False
        )
        
        
        age_marks = {i: {'label': str(i), 'style': {'color': '#1a2d40'}} 
                   for i in range(0, 101, 10)}
        
        return (
            df.to_json(date_format='iso', orient='split'),
            {'display': 'block'},
            df['last_update'].min(),
            df['last_update'].max(),
            age_marks
        )
    except Exception as e:
        print(f"Ошибка при чтении файла: {e}")
        return dash.no_update, {'display': 'none'}, dash.no_update, dash.no_update, dash.no_update


@app.callback(
    Output("vital-signs-plot", "figure"),
    Output("patient-distribution", "figure"),
    Output("hr-distribution", "figure"),
    Output("advanced-stats", "children"),
    Input("gender-filter", "value"),
    Input("condition-filter", "value"),
    Input("age-slider", "value"),
    Input("date-range", "start_date"),  
    Input("date-range", "end_date"),
    Input('stored-data', 'data')
)
def update_dashboard(selected_gender, selected_condition, age_range, 
                    start_date, end_date, stored_data):
    if stored_data is None:
        return dash.no_update, dash.no_update, dash.no_update, dash.no_update
    
    df = pd.read_json(io.StringIO(stored_data), orient='split')
    
    
    df['last_update'] = pd.to_datetime(
        df['last_update'], 
        format='%Y-%m-%dT%H:%M:%S.%f', 
        errors='coerce'
    )
    
    
    filtered_df = df[
        (df["last_update"] >= pd.to_datetime(start_date)) &
        (df["last_update"] <= pd.to_datetime(end_date)) &
        (df["age"] >= age_range[0]) &
        (df["age"] <= age_range[1])
    ]

    if selected_gender != "All":
        filtered_df = filtered_df[filtered_df["gender"] == selected_gender]
    if selected_condition != "All":
        filtered_df = filtered_df[filtered_df["condition"] == selected_condition]

    # График 1: Динамика сердечного ритма
    fig1 = px.scatter(
        filtered_df,
        x="last_update",
        y="heart_rate",
        color="condition",
        size="age",
        hover_data=["blood_pressure"],
        title="Динамика сердечного ритма",
        template="plotly_white",
        color_discrete_map={
            'Stable': '#2ecc71',
            'Critical': '#e74c3c',
            'Recovering': '#3498db'
        }
    ).update_layout(
        plot_bgcolor='rgba(0,0,0,0)',
        paper_bgcolor='rgba(0,0,0,0)',
        hovermode='x unified'
    )

    # График 2: Распределение состояний
    fig2 = px.pie(
        filtered_df,
        names="condition",
        hole=0.5,
        title="Распределение состояний",
        color="condition",
        color_discrete_map={
            'Stable': '#2ecc71',
            'Critical': '#e74c3c',
            'Recovering': '#3498db'
        }
    ).update_traces(
        textposition='inside', 
        textinfo='percent+label'
    )

    # График 3: Распределение пульса
    fig3 = px.histogram(
        filtered_df,
        x="heart_rate",
        nbins=20,
        color="gender",
        title="Распределение пульса",
        color_discrete_map={
            'Male': '#2980b9',
            'Female': '#e67e22'
        }
    ).update_layout(
        bargap=0.1,
        showlegend=False
    )

    # Статистика
    if not filtered_df.empty:
        stats = [
            html.H4("📊 Статистика выборки", style={'marginBottom': '15px'}),
            html.Div([
                html.Div([
                    html.P("Всего пациентов", style={'opacity': 0.8}),
                    html.H3(f"{len(filtered_df)}", style={'color': '#2ecc71'})
                ], style={'width': '24%', 'display': 'inline-block'}),
                
                html.Div([
                    html.P("Средний возраст", style={'opacity': 0.8}),
                    html.H3(f"{filtered_df['age'].mean():.1f} лет", style={'color': '#3498db'})
                ], style={'width': '24%', 'display': 'inline-block'}),
                
                html.Div([
                    html.P("Средний пульс", style={'opacity': 0.8}),
                    html.H3(f"{filtered_df['heart_rate'].mean():.1f} уд/мин", style={'color': '#e67e22'})
                ], style={'width': '24%', 'display': 'inline-block'}),
                
                html.Div([
                    html.P("Критических случаев", style={'opacity': 0.8}),
                    html.H3(f"{sum(filtered_df['condition'] == 'Critical')}", style={'color': '#e74c3c'})
                ], style={'width': '24%', 'display': 'inline-block'})
            ])
        ]
    else:
        stats = html.P("Нет данных для отображения", style={'textAlign': 'center'})

    return fig1, fig2, fig3, stats

if __name__ == "__main__":
      app.run(debug=True, port=8051)